# Capstone Project - The Battle of the Neighborhoods (Week 1)
### For: Applied Data Science Capstone (IBM/Coursera)

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a gym. Specifically, this report will be targeted to stakeholders interested in opening a **Martial Arts Gym** in **New York, NY**.

Since there are lots of gyms in NYC we will try to detect **locations that are not already crowded with gyms**. We are also particularly interested in **areas with no Martial Arts Gyms in the vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use data science techniques to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:
* number of existing gyms in the neighborhood (any type of gym)
* number of and distance to Martial Arts Gyms in the neighborhood, if any
* distance of neighborhood from city center

Regularly spaced grids of locations were used, centered around city center, to define our neighborhoods.

The following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of Gyms and their type and location in every neighborhood will be obtained using the **Foursquare API**
* coordinates of the NYC center will be obtained using **Google Maps API geocoding**

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around the NYC city center.

Let's first find the latitude & longitude of the NYC city center, using specific, well known address and Google Maps geocoding API.

In [2]:
# Coordinates

In [70]:
import os
google_api_key=os.environ.get('google_geocode_apik')

In [71]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Woodside, NY'
nyc_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, nyc_center))

Coordinate of Woodside, NY: [40.7532952, -73.9068973]


Now let's create a grid of area candidates, equaly spaced, centered around the city center and within ~12km from Woodside, NY. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in a Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on a Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [72]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('NYC center longitude={}, latitude={}'.format(nyc_center[1], nyc_center[0]))
x, y = lonlat_to_xy(nyc_center[1], nyc_center[0])
print('NYC center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('NYC center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
NYC center longitude=-73.9068973, latitude=40.7532952
NYC center UTM X=-5815056.576617461, Y=9857010.472669667
NYC center longitude=-73.90689729999957, latitude=40.7532951999989


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [73]:
nyc_center_x, nyc_center_y = lonlat_to_xy(nyc_center[1], nyc_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = nyc_center_x - 6000
x_step = 600
y_min = nyc_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(nyc_center_x, nyc_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [74]:
#!pip install folium

import folium

In [75]:
map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.Marker(nyc_center, popup='NYC Center - Woodside').add_to(map_nyc)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_nyc) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_nyc)
    #folium.Marker([lat, lon]).add_to(map_nyc)
map_nyc

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~12km from the center of NYC. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [76]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, nyc_center[0], nyc_center[1])
print('Reverse geocoding check')
print('-----------------------') 
print('Address of [{}, {}] is: {}'.format(nyc_center[0], nyc_center[1], addr))


Reverse geocoding check
-----------------------
Address of [40.7532952, -73.9068973] is: Northern Blvd, Queens, NY 11377, USA


In [77]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
#     address = address.replace(', United States', '') # We don't need country part of address
    addresses.append(address)
    print(address)
#     print(' .', end='')
print(' done.')

Obtaining location addresses: 98-50 50th Ave, Corona, NY 11368, USA
101-40 Nicolls Ave, Flushing, NY 11368, USA
4102 103rd St, Corona, NY 11368, USA
37-55 104th St, Flushing, NY 11368, USA
34-02 106th St, Flushing, NY 11368, USA
32-16 107th St, Flushing, NY 11369, USA
108-46 Ditmars Blvd, Flushing, NY 11369, USA
94-1 57th Ave, Queens, NY 11373, USA
52-7 Junction Blvd, Corona, NY 11368, USA
96-11 Alstyne Ave, Flushing, NY 11368, USA
41 97th Pl, Corona, NY 11368, USA
37-64 100th St, Flushing, NY 11368, USA
101-08 35th Ave, Flushing, NY 11368, USA
102-3 Northern Blvd, East Elmhurst, NY 11369, USA
103-2 Astoria Blvd, East Elmhurst, NY 11369, USA
104-07 27th Ave, Flushing, NY 11369, USA
104 Ditmars Blvd, East Elmhurst, NY 11369, USA
Woodhaven Blvd /Wetherole St, Queens, NY 11374, USA
91- 20 57th Ave, Queens, NY 11373, USA
90-60 53rd Ave, Flushing, NY 11373, USA
93-01 49th Ave, Queens, NY 11373, USA
94-16 42nd Ave, Flushing, NY 11373, USA
95-30 Roosevelt Ave, Jackson Heights, NY 11372, USA
3

Laurel Hill Blvd, Woodside, NY 11377, USA
53-15 Queens Blvd, Woodside, NY 11377, USA
52-29 Skillman Ave, Flushing, NY 11377, USA
52-16 Barnett Ave, Sunnyside, NY 11104, USA
50-49 Newtown Rd, Woodside, NY 11377, USA
48-5 Broadway, Woodside, NY 11377, USA
46-12 30th Rd, Long Island City, NY 11103, USA
43-20 28th Ave, Astoria, NY 11103, USA
41-09 25th Ave, Long Island City, NY 11103, USA
23-33 37th St, Long Island City, NY 11105, USA
22-25 33rd St, Long Island City, NY 11105, USA
21-19 29th St, Long Island City, NY 11105, USA
20-10 26th St, Long Island City, NY 11105, USA
21-20 16th Ave, Queens, NY 11105, USA
52-17 Grand Ave, Maspeth, NY 11378, USA
56-01 Maspeth Ave, Maspeth, NY 11378, USA
55-64 56th St, Maspeth, NY 11378, USA
53 Queens Midtown Expy, Woodside, NY 11377, USA
Laurel Hill Blvd, Woodside, NY 11377, USA
48-58 49th St, Woodside, NY 11377, USA
45-44 50th St, Flushing, NY 11377, USA
49-07 43rd Ave, Woodside, NY 11377, USA
39-24 49th St, Long Island City, NY 11104, USA
4818 48th S

In [78]:
addresses[-50:]

['1950 35th Ave, Queens, NY 11106, USA',
 '33-38 14th St, Long Island City, NY 11106, USA',
 '11-40 31st Dr, Queens, NY 11106, USA',
 '3002 Vernon Blvd, Astoria, NY 11102, USA',
 '2-23 27th Ave, Astoria, NY 11102, USA',
 '1-09 26th Ave, Long Island City, NY 11102, USA',
 '360 Kingsland Ave, Brooklyn, NY 11222, USA',
 '5415 35th St, Long Island City, NY 11101, USA',
 '52-01 29th St, Long Island City, NY 11101, USA',
 '47-60 29th St, Long Island City, NY 11101, USA',
 '27-20 Skillman Ave, Long Island City, NY 11101, USA',
 '42 Rd/Hunter St, Queens, NY 11101, USA',
 '2506 41st Ave, Long Island City, NY 11101, USA',
 '39-01 22nd St, Long Island City, NY 11101, USA',
 '13-01 37th Ave, Long Island City, NY 11101, USA',
 '11-2 35th Ave, Astoria, NY 11106, USA',
 '33-11 9th St, Queens, NY 11106, USA',
 '150-41 Vernon Blvd, Queens, NY 11106, USA',
 '104 Astoria Blvd, Astoria, NY 11102, USA',
 '26-40 1st St, Long Island City, NY 11102, USA',
 '403 Greenpoint Ave, Brooklyn, NY 11222, USA',
 '540 

Looking good. Let's now place all this into a Pandas dataframe.

In [79]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"98-50 50th Ave, Corona, NY 11368, USA",40.741686,-73.863013,-5.816857e+06,9.851295e+06,5992.495307
1,"101-40 Nicolls Ave, Flushing, NY 11368, USA",40.745222,-73.862872,-5.816257e+06,9.851295e+06,5840.376700
2,"4102 103rd St, Corona, NY 11368, USA",40.748759,-73.862731,-5.815657e+06,9.851295e+06,5747.173218
3,"37-55 104th St, Flushing, NY 11368, USA",40.752296,-73.862591,-5.815057e+06,9.851295e+06,5715.767665
4,"34-02 106th St, Flushing, NY 11368, USA",40.755833,-73.862450,-5.814457e+06,9.851295e+06,5747.173218
5,"32-16 107th St, Flushing, NY 11369, USA",40.759370,-73.862309,-5.813857e+06,9.851295e+06,5840.376700
6,"108-46 Ditmars Blvd, Flushing, NY 11369, USA",40.762907,-73.862168,-5.813257e+06,9.851295e+06,5992.495307
7,"94-1 57th Ave, Queens, NY 11373, USA",40.736475,-73.867249,-5.817757e+06,9.851814e+06,5855.766389
8,"52-7 Junction Blvd, Corona, NY 11368, USA",40.740011,-73.867109,-5.817157e+06,9.851814e+06,5604.462508
9,"96-11 Alstyne Ave, Flushing, NY 11368, USA",40.743547,-73.866969,-5.816557e+06,9.851814e+06,5408.326913


...and let's now save/persist this data into local file.

In [80]:
df_locations.to_pickle('./locations.pkl')

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on gyms in each neighborhood.

We're interested in venues in 'Gyms / Fitness' category, and we'll make sure to detect and include all the subcategories of specific 'Martial Arts Gym' category, as we need info on Martial Arts Dojos in the neighborhood.

Foursquare credentials are defined bellow.

In [81]:
CLIENT_ID = os.environ.get('foursq_client_id') # your Foursquare ID
CLIENT_SECRET = os.environ.get('foursq_client_secret') # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [82]:
#Get list of venue categories around New York
LIMIT = 1000 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={nyc_center[0]},{nyc_center[1]}&radius={radius}&limit={LIMIT}'

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edb202b1a4b0a001bb85566'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Woodside',
  'headerFullLocation': 'Woodside, Queens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 157,
  'suggestedBounds': {'ne': {'lat': 40.76229520900001,
    'lng': -73.89503871315469},
   'sw': {'lat': 40.744295190999985, 'lng': -73.9187558868453}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a466fc5610f0447488800e2',
       'name': 'Xi’an Famous Foods Central Kitchen',
       'location': {'address': '55-01 37th Ave',
        'lat': 40.750899,
        'lng': -73.906499,
        'labeledLatLngs': [{'l

In [83]:
#find Gym category ID
from pprint import pprint

for i in results['response']:
    print(i)

print(len(results['response']['groups']))

gymdictlst=[]
for i in results['response']['groups'][0]['items']:
#     pprint(i['venue']['categories'][0])
    if 'Gym' in i['venue']['categories'][0]['pluralName']:
        gymdictlst.append(i['venue']['categories'][0])
    
gymdictlst

suggestedFilters
headerLocation
headerFullLocation
headerLocationGranularity
totalResults
suggestedBounds
groups
1


[{'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d176941735',
  'name': 'Gym',
  'pluralName': 'Gyms',
  'shortName': 'Gym',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary': True},
 {'id': '4bf58dd8d48988d175941735',
  'name': 'Gym / Fitness Center',
  'pluralName': 'Gyms or Fitness Centers',
  'shortName': 'Gym / Fitness',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
   'suffix': '.png'},
  'primary

In [84]:
for i in results['response']['groups'][0]['items']:
    pprint(i['venue']['categories'][0]['pluralName'])

'Chinese Restaurants'
'Nightclubs'
'Juice Bars'
'Donut Shops'
'Pizza Places'
'American Restaurants'
'Health Food Stores'
'Italian Restaurants'
'Bars'
'Convenience Stores'
'Taco Places'
'German Restaurants'
'Nightclubs'
'Martial Arts Dojos'
'Butchers'
'Gyms or Fitness Centers'
'Parks'
'Arepa Restaurants'
'South American Restaurants'
'Bakeries'
'Chinese Restaurants'
'Chinese Restaurants'
'Bars'
'Himalayan Restaurants'
'Pizza Places'
'Sushi Restaurants'
'Thai Restaurants'
'Bakeries'
'Yoga Studios'
'Gyms or Fitness Centers'
'Ramen Restaurants'
'Fast Food Restaurants'
'Pubs'
'Coffee Shops'
'Bars'
'American Restaurants'
'Latin American Restaurants'
'Bars'
'Burger Joints'
'Eastern European Restaurants'
'Bakeries'
'Thai Restaurants'
'Parks'
'Middle Eastern Restaurants'
'Fast Food Restaurants'
'French Restaurants'
'Fried Chicken Joints'
'Pet Stores'
'Bagel Shops'
'Pubs'
'Thai Restaurants'
'American Restaurants'
'Coffee Shops'
'Bars'
'Sushi Restaurants'
'Rental Car Locations'
'American Restauran

In [85]:
fsgymid='4bf58dd8d48988d175941735'
martial_arts_fs_id = '4bf58dd8d48988d101941735'

In [90]:
# Category IDs on Foursquare web site (https://developer.foursquare.com/docs/resources/categories)

gym_category = fsgymid # 'Root' category for all gym-related venues

martial_arts_cat = martial_arts_fs_id

def is_gym(categories, specific_filter=None):
    gym_words = ['gym', 'Gym', 'gyms', 'Gyms']
    gym = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in gym_words:
            if r in category_name:
                gym = True
#         if 'Martial Arts' in category_name:
#             gym = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            gym = True
    return gym, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [92]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_gyms(lats, lons):
    gyms = {}
    ma_gyms = {}
    location_gyms = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any gym (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, gym_category, CLIENT_ID, CLIENT_SECRET, radius=350, limit=100)
        area_gyms = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_g, is_ma = is_gym(venue_categories, specific_filter=martial_arts_cat)
            if is_g:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                gym = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_ma, x, y)
                if venue_distance<=300:
                    area_gyms.append(gym)
                gyms[venue_id] = gym
                if is_ma:
                    ma_gyms[venue_id] = gym
        location_gyms.append(area_gyms)
        print(' .', end='')
    print(' done.')
    return gyms, ma_gyms, location_gyms

# Try to load from local file system in case we did this before
gyms = {}
ma_gyms = {}
location_gyms = []
loaded = False
try:
    with open('gyms_350.pkl', 'rb') as f:
        gyms = pickle.load(f)
    with open('ma_gyms_350.pkl', 'rb') as f:
        ma_gyms = pickle.load(f)
    with open('location_gyms_350.pkl', 'rb') as f:
        location_gyms = pickle.load(f)
    print('Gym data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    gyms, ma_gyms, location_gyms = get_gyms(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('gyms_350.pkl', 'wb') as f:
        pickle.dump(gyms, f)
    with open('ma_gyms_350.pkl', 'wb') as f:
        pickle.dump(ma_gyms, f)
    with open('location_gyms_350.pkl', 'wb') as f:
        pickle.dump(location_gyms, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [93]:
import numpy as np

print('Total number of gyms:', len(gyms))
print('Total number of Martial Arts Dojos:', len(ma_gyms))
print('Percentage of Martial Arts Dojos: {:.2f}%'.format(len(ma_gyms) / len(gyms) * 100))
print('Average number of gyms in neighborhood:', np.array([len(r) for r in location_gyms]).mean())

Total number of gyms: 258
Total number of Martial Arts Dojos: 33
Percentage of Martial Arts Dojos: 12.79%
Average number of gyms in neighborhood: 1.282967032967033


In [94]:
print('List of all gyms')
print('-----------------------')
for r in list(gyms.values())[:10]:
    print(r)
print('...')
print('Total:', len(gyms))

List of all gyms
-----------------------
('5032bb1ae4b0a81e08dcea88', 'Turbo Jam', 40.743348800072575, -73.86022376650897, 'Corona, NY 11368, United States', 305, False, -5816563.824867777, 9850943.669013994)
('5032b948e4b0feaf1a5815a0', 'Team Beach Body', 40.743255615234375, -73.8601303100586, 'Corona, NY 11368, United States', 318, False, -5816579.255822855, 9850931.140623344)
('4f2854eee4b055df286014a5', 'Xtreme Fitness', 40.74964386760678, -73.86522963352094, 'Jackson Heights, NY 11372, United States', 164, False, -5815516.296461998, 9851621.291540632)
('5326e732498eed3bc8151eae', 'Xtreme Fitness Gym', 40.74965878060291, -73.86521458625793, '100-12 Roosevelt Ave, Corona, NY 11368, United States', 163, False, -5815513.71020383, 9851619.428230746)
('4e30c3c381dc65de51a337b5', 'United TaeKwonDo Center Corona School', 40.75028264745698, -73.8656242914668, '40-10 National St, Corona, NY 11368, United States', 97, True, -5815409.56168368, 9851675.43126088)
('4d408328915a37040b38167f', 'L

In [95]:
print('List of Martial Arts Dojos')
print('---------------------------')
for r in list(ma_gyms.values())[:10]:
    print(r)
print('...')
print('Total:', len(ma_gyms))

List of Martial Arts Dojos
---------------------------
('4e30c3c381dc65de51a337b5', 'United TaeKwonDo Center Corona School', 40.75028264745698, -73.8656242914668, '40-10 National St, Corona, NY 11368, United States', 97, True, -5815409.56168368, 9851675.43126088)
('55f4589c498e0794ff1956f1', 'Tristate Martial Arts Institute', 40.73549848213117, -73.87281152101782, '85-04 Queens Blvd, Elmhurst, NY 11373, United States', 246, True, -5817943.655441224, 9852526.696271645)
('4e75002d8998ed82a4546a74', 'DoMA TaeKwonDo JH', 40.74891756570473, -73.87146205128845, 'New York, United States', 347, True, -5815663.591150319, 9852420.969462162)
('4e51240ce4cdc76dc9c85903', 'DoM\tA Tkd', 40.749122619628906, -73.87116241455078, 'New York, United States', 37, True, -5815627.674681875, 9852383.388243847)
('5dfa55ccf3df6f0008f5ff42', 'United Xtreme Martial Arts', 40.7380581, -73.8816611, '81-15 Queens Blvd, Elmhurst, NY 11373, United States', 289, True, -5817543.790611963, 9853680.920569172)
('4c802635d4

In [96]:
print('Gyms around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_gyms[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Gyms around location {}: {}'.format(i+1, names))

Gyms around location
---------------------------
Gyms around location 101: Urban Jungle
Gyms around location 102: 
Gyms around location 103: 
Gyms around location 104: New Self Image, Blink Fitness, S.Y. Kims Tae Kwan Doe Center
Gyms around location 105: 
Gyms around location 106: 
Gyms around location 107: 
Gyms around location 108: 
Gyms around location 109: 
Gyms around location 110: Shorehaven Gym


Let's now see all the collected gyms in our area of interest on map, and let's also show Martial Arts Dojos in a  different color.

In [97]:
map_nyc = folium.Map(location=nyc_center, zoom_start=13)
folium.Marker(nyc_center, popup='NYC Center').add_to(map_nyc)
for res in gyms.values():
    lat = res[2]; lon = res[3]
    is_ma = res[6]
    color = 'red' if is_ma else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_nyc)
map_nyc

So now we have all the gyms in area within few kilometers from the NYC center, and we know which ones are Martial Arts Dojos. We also know which gyms exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Martial Arts Dojo!